# Network analysis
[original code](https://github.com/joelgrus/data-science-from-scratch/blob/master/code/network_analysis.py)

In [1]:
from network_analysis import *

### Betweenness centrality
Identify people who frequently are on the shortest paths between pairs.
Not really used for large networks given its difficulty to compute shortest paths.

In [5]:
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [12]:
# give each user a friends list
for user in users:
    user["friends"] = []
    
# and populate it
for i, j in friendships:
    # this works because users[i] is the user whose id is i
    users[i]["friends"].append(users[j]) # add i as a friend of j
    users[j]["friends"].append(users[i]) # add j as a friend of i   

Calculates the shortest paths from a given user.

In [33]:

def shortest_paths_from(from_user):
    
    # a dictionary from "user_id" to *all* shortest paths to that user
    shortest_paths_to = { from_user["id"] : [[]] }

    # a queue of (previous user, next user) that we need to check.
    # starts out with all pairs (from_user, friend_of_from_user)
    frontier = deque((from_user, friend)
                     for friend in from_user["friends"])

    # keep going until we empty the queue
    while frontier: 

        prev_user, user = frontier.popleft() # take from the beginning
        user_id = user["id"]

        # the fact that we're pulling from our queue means that
        # necessarily we already know a shortest path to prev_user
        paths_to_prev = shortest_paths_to[prev_user["id"]]
        paths_via_prev = [path + [user_id] for path in paths_to_prev]
        
        # it's possible we already know a shortest path to here as well
        old_paths_to_here = shortest_paths_to.get(user_id, [])
        
        # what's the shortest path to here that we've seen so far?
        if old_paths_to_here:
            min_path_length = len(old_paths_to_here[0])
        else:
            min_path_length = float('inf')
                
        # any new paths to here that aren't too long
        new_paths_to_here = [path_via_prev
                             for path_via_prev in paths_via_prev
                             if len(path_via_prev) <= min_path_length
                             and path_via_prev not in old_paths_to_here]
        
        shortest_paths_to[user_id] = old_paths_to_here + new_paths_to_here
        
        # add new neighbors to the frontier
        frontier.extend((user, friend)
                        for friend in user["friends"]
                        if friend["id"] not in shortest_paths_to)

    return shortest_paths_to

paths_from_0 = shortest_paths_from(users[0])
print "List of paths from id=0: " 
for i in paths_from_0.iteritems():
    print i

List of paths from id=0: 
(0, [[]])
(1, [[1]])
(2, [[2]])
(3, [[1, 3], [2, 3]])
(4, [[1, 3, 4], [2, 3, 4]])
(5, [[1, 3, 4, 5], [2, 3, 4, 5]])
(6, [[1, 3, 4, 5, 6], [2, 3, 4, 5, 6]])
(7, [[1, 3, 4, 5, 7], [2, 3, 4, 5, 7]])
(8, [[1, 3, 4, 5, 6, 8], [2, 3, 4, 5, 6, 8], [1, 3, 4, 5, 7, 8], [2, 3, 4, 5, 7, 8]])
(9, [[1, 3, 4, 5, 6, 8, 9], [2, 3, 4, 5, 6, 8, 9], [1, 3, 4, 5, 7, 8, 9], [2, 3, 4, 5, 7, 8, 9]])


In [72]:
for user in users:
    user["shortest_paths"] = shortest_paths_from(user)

for user in users:
    user["betweenness_centrality"] = 0.0

for source in users:
    source_id = source["id"]
    for target_id, paths in source["shortest_paths"].iteritems():
        if source_id < target_id:   # don't double count
            num_paths = len(paths)  # how many shortest paths?
            contrib = 1 / num_paths # contribution to centrality
            for path in paths:
                for id in path:
                    if id not in [source_id, target_id]:
                        users[id]["betweenness_centrality"] += contrib

                        

for u in users:
    print "%d - %d" %  (u['id'], u["betweenness_centrality"])

0 - 0
1 - 0
2 - 0
3 - 8
4 - 9
5 - 8
6 - 0
7 - 0
8 - 2
9 - 0


### Closeness centrality

It computes the farness of each user, which is the sum of the lengths of their shortest paths.

In [69]:
def farness(user):
    """the sum of the lengths of the shortest paths to each other user"""
    return sum(len(paths[0]) 
               for paths in user["shortest_paths"].values())

for user in users:
    user["closeness_centrality"] = 1.0 / farness(user)
    

for u in users:
    print "%d - %.4f" %  (u['id'], u["closeness_centrality"])

0 - 0.0294
1 - 0.0370
2 - 0.0370
3 - 0.0455
4 - 0.0500
5 - 0.0500
6 - 0.0417
7 - 0.0417
8 - 0.0357
9 - 0.0278


### Eigenvector centrality
More frequently used.
It gives bigger numbers to users connected to people who are more central. Easier to compute in larger networks.

In [78]:
# if user i and user j are friends, Aij = 1 else Aij=0
adjacency_matrix =make_matrix(9,9, entry_fn)
for row in adjacency_matrix:
    print row

[0, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 1, 0]


find_eigenvector: 
it starts by assigning each node a random centrality.  It repeats the following 2 steps until the process converges:
    1. give each node a new centrality score that equals the sum of its neighbors' old centrality scores.
    2. rescale the vector of centralities to have magnitude 1.

In [84]:
eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)
for i, c in enumerate(eigenvector_centralities):
    print "%d - %.4f" %  (i, c)

0 - 0.3881
1 - 0.5176
2 - 0.5176
3 - 0.4749
4 - 0.2315
5 - 0.1425
6 - 0.0743
7 - 0.0743
8 - 0.0557


Given the example network is not too big, the results are not very accurate.

## Directed graphs


### Page rank


In [86]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2), (2, 1), (1, 3),
                (2, 3), (3, 4), (5, 4), (5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

for user in users:
    user["endorses"] = []       # add one list to track outgoing endorsements
    user["endorsed_by"] = []    # and another to track endorsements
    
for source_id, target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])

In [93]:
users_ranked = page_rank(users)
for i, pr in users_ranked.iteritems():
    print "%d - %.5f" %  (i, pr)

0 - 0.04046
1 - 0.04492
2 - 0.04492
3 - 0.04046
4 - 0.06785
5 - 0.04344
6 - 0.03346
7 - 0.03346
8 - 0.04344
9 - 0.03346
